In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data
df = pd.read_csv("laptop_data.csv")

In [3]:
#Lets filter the unwanted data,like monitors, tablet extra
df = df[df['CPU Speed'].str.contains('GHz')]

In [4]:
# Lets make the review feature as numeric
df['No. Reviews'] = df['No. Reviews'].str.replace(',','').astype(int)

In [5]:
#Removed irrelavant data and converted it to numeric
df['Display size'] = df[df['Display size'].str.contains('in')]['Display size'].str.replace('in','')
df = df.dropna()
df.drop(df[df['Display size'].str.contains('W')].index , inplace =True)
df['Display size'] = df['Display size'].astype(float)

In [6]:
# As there are mulitple values for CPU speed, its hard to create dummy columns. So I changed it to numeric
df['CPU Speed'] = df['CPU Speed'].str.replace('GHz','').astype(float)

In [7]:
#Dropped duplicate rows
df.drop(df[df['RAM'].str.contains('-')][1:].index, inplace = True)

In [8]:
#Cleaning the RAM feature
df.RAM = df.RAM.str.replace('GB','')
df.RAM = df.RAM.str.replace('TB','')

df.RAM = df.RAM.replace( '-' , np.NAN)
df.RAM = df.RAM.fillna(df['RAM'].mode()[0]).astype(int)

In [9]:
#Cleaning the ratings column
df.Rating = df['Rating'].str[:3].astype(float)

In [10]:
#Fixing the storage types
df['Storage type'] = df['Storage type'].str.replace('Solid State Drive','SSD')
df['Storage type'] = df['Storage type'].apply(lambda x : 'SSD' if ('ssd' in x.lower()) else x )

#Fixed the EMMC
df['Storage type'] = df['Storage type'].apply(lambda x : 'EMMC' if ('emmc' in x.lower()) else x )
df['Storage type'] = df['Storage type'].apply(lambda x : 'EMMC' if ('Embedded MultiMediaCard' in x) else x )

#Fixed the HDD
df['Storage type'] = df['Storage type'].apply(lambda x : 'HDD' if ('hdd' in x.lower()) else x )

#Replacing the rest of the data with others
df['Storage type'] = df['Storage type'].replace('Mechanical Hard Drive','Others').replace('Hybrid Drive','Others')
df['Storage type'] = df['Storage type'].replace('-','Others').replace('Integrated','Others').replace('Hybrid','Others').replace('SDD','Others')
df['Storage type'] = df['Storage type'].replace('64gb','Others').replace('Flash','Others').replace('sshd','Others').replace('USB','Others')